In [3]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import filters
import scipy.ndimage as sci_image
from skimage import measure as meas
import matplotlib.pyplot as plt
from scipy import signal as sci_signal
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import os
import warnings
warnings.filterwarnings("ignore")
import supported_functions as sf

In [9]:
def detect_letters(image):
    image_resized = cv2.resize(image,(24,42))
    image_resized_grayscale = cv2.cvtColor(src=image_resized,code=cv2.COLOR_BGR2GRAY) / 255.0
    ctr = 0
    corr_matrix_arr = []
    for i in sf.create_templates():
        corr_matrix = sci_signal.correlate2d(i,image_resized_grayscale).max()
        corr_matrix_arr.append(corr_matrix)
    pos = np.argmax(corr_matrix_arr)
    det = sf.identify_letter(pos)
    return pos

In [10]:
detect_letters(cv2.imread("data/D.bmp"))

13